In [88]:
# ╔═══════════════════════════════════════════════════════════════════════╗
# ║             INTERACTIVE DATASET & PATH CONFIGURATOR                   ║
# ╚═══════════════════════════════════════════════════════════════════════╝
import ipywidgets as wd
from pathlib import Path
from IPython.display import display, clear_output
import os

# --- Default or Patch Selections ---
# The script will attempt to use this list first for examples.
DEFAULT_PATCH_SELECTIONS = [
    (2020, "29TPF", "2020_29TPF_patch_20_15"),
    (2021, "29TQG", "2021_29TQG_patch_29_23"),
    (2022, "30TVM", "2022_30TVM_patch_13_29"),
    (2023, "29TQG", "2023_29TQG_patch_10_21"),
    (2024, "30TUM", "2024_30TUM_patch_20_15"),
    (2024, "30TUM", "2024_30TUM_patch_00_16"),
    (2024, "30TUK", "2024_30TUK_patch_00_00"),
]

def find_available_patches(base_path, ideal_selections, num_to_find=7):
    """
    Checks if the ideal patches exist. If not, scans the disk for alternatives.
    """
    all_ideal_exist = True
    for year, tile, stem in ideal_selections:
        patch_file = base_path / str(year) / tile / f"{stem}.nc"
        if not patch_file.exists():
            all_ideal_exist = False
            break
    
    if all_ideal_exist:
        print("✅ Default example patches found and loaded.")
        return ideal_selections

    print("⚠️  Some default example patches were not found. Searching for alternatives on disk...")
    found_patches = []
    for path in base_path.rglob("*.nc"):
        try:
            stem = path.stem
            tile = path.parent.name
            year = int(path.parent.parent.name)
            
            if stem.startswith(f"{year}_{tile}"):
                found_patches.append((year, tile, stem))
                if len(found_patches) >= num_to_find:
                    break
        except (ValueError, IndexError):
            continue
            
    if found_patches:
        print(f"✅ Found {len(found_patches)} available patches to use as examples.")
    
    return found_patches

title_html = wd.HTML("<h3>1. Configure Project Paths</h3>"
                     "<p>Point the <b>Root Path</b> to the main dataset folder "
                     "<code>S4A-CyL_Dataset_2020-2024</code> and press the button to validate.</p>")

default_root = Path.home() / "S4A-CyL_Dataset_2020-2024"
root_path_text = wd.Text(
    value=str(default_root),
    placeholder='e.g., /home/user/datasets/S4A-CyL_Dataset_2020-2024',
    description='Root Path:',
    style={'description_width': 'initial'},
    layout=wd.Layout(width='90%')
)

output_path_text = wd.Text(
    value=str(Path.home() / "Downloads" / "S4A_CyL_diagnostics"),
    placeholder='e.g., /path/to/save/outputs',
    description='Output Dir:',
    style={'description_width': 'initial'},
    layout=wd.Layout(width='90%')
)

confirm_button = wd.Button(
    description="Verify & Set Paths",
    button_style='success',
    tooltip='Validate paths and load them as global variables',
    icon='check'
)
status_output = wd.Output()

def on_confirm_button_clicked(b):
    global BASE_NAS, OUT_DIR, CSV_CANDIDATES, PATCH_SELECTIONS
    with status_output:
        clear_output(wait=True)
        
        root_path = Path(root_path_text.value.strip())
        out_path = Path(output_path_text.value.strip())
        
        print(f"Verifying root path: {root_path}")
        if not root_path.exists() or not root_path.is_dir():
            print(f"❌ ERROR: Root path does not exist or is not a directory.")
            return
        print(f"✅ Root path found.")
        
        BASE_NAS = root_path / "netcdf_data"
        CROP_MAP_PATH = root_path / "mapping_tables" / "crop_id_mapping.csv"
        
        if not BASE_NAS.exists():
            print(f"❌ ERROR: The 'netcdf_data' directory was not found inside the root path.")
            print(f"   Checked here: {BASE_NAS}")
            return
        print(f"✅ 'netcdf_data' directory found.")

        if not CROP_MAP_PATH.exists():
            print(f"❌ ERROR: The 'crop_id_mapping.csv' file was not found.")
            print(f"   Checked here: {CROP_MAP_PATH}")
            return
        print(f"✅ 'crop_id_mapping.csv' found.")
        
        OUT_DIR = out_path
        CSV_CANDIDATES = [CROP_MAP_PATH]
        
        PATCH_SELECTIONS = find_available_patches(BASE_NAS, DEFAULT_PATCH_SELECTIONS)
        
        if not PATCH_SELECTIONS:
            print("\n❌ CRITICAL ERROR: No NetCDF patches (.nc) were found in the 'netcdf_data' folder.")
            print("   Please ensure the data is downloaded and located correctly within the specified root path.")
            return
            
        OUT_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✅ Output directory is ready: {OUT_DIR}")
        print("\n🎉 Configuration complete! You can now run the patch viewer cell below.")

confirm_button.on_click(on_confirm_button_clicked)
ui = wd.VBox([
    title_html,
    root_path_text,
    output_path_text,
    wd.HBox([confirm_button]),
    status_output
])

display(ui)

In [87]:
# ╔═════════════ PATCH VIEWER v18 · 3 overlay styles ══════════════╗
# NOTE: Make sure you have successfully run the configuration cell above
# before executing this one.

import os, netCDF4, xarray as xr, numpy as np, matplotlib.pyplot as plt
import matplotlib as mpl, matplotlib.patches as mpatches
import pandas as pd, ipywidgets as wd
from pathlib import Path
from IPython.display import display, clear_output

# NOTE: This cell requires the scikit-image library for advanced color correction.
# pip install scikit-image
from skimage import exposure

# --- Check if config variables exist ---
try:
    _ = BASE_NAS
    _ = PATCH_SELECTIONS
except NameError:
    raise NameError("Configuration variables not found. Please run the setup cell above first.")

# --- Load crop map ---
crop_map = {}
for p in CSV_CANDIDATES:
    if p.exists():
        df = pd.read_csv(p, header=None, names=["Crop", "Code"])
        crop_map = {int(r.Code): r.Crop for r in df.itertuples()}
        print(f"✓ Crop map loaded ({len(crop_map)}) from {p}")
        break
if not crop_map:
    print("⚠️ WARNING: Could not load the crop map. Labels will show as numeric codes.")

# --- Utility functions ---
def open_ds(root, grp):
    return xr.open_dataset(xr.backends.NetCDF4DataStore(root[grp]), mask_and_scale=False)

def load_patch_disk(y, tile, stem):
    f = BASE_NAS / str(y) / tile / f"{stem}.nc"
    root = netCDF4.Dataset(f, "r")
    bands = {g: open_ds(root, g) for g in root.groups if g.startswith("B")}
    parc  = open_ds(root, "parcels")
    lab   = open_ds(root, "labels")
    return bands, parc, lab

def stretch(a, p1=2, p2=98, gamma=1.0):
    """Stretches the input array's contrast and applies gamma correction."""
    lo, hi = np.nanpercentile(a, [p1, p2])
    a_stretched = np.clip((a - lo) / (hi - lo + 1e-9), 0, 1)
    if gamma != 1.0:
        a_stretched = np.power(a_stretched, 1 / gamma)
    return a_stretched

def resize_to(a, tgt):
    if a.shape == tgt.shape: return a
    fy = tgt.shape[0] // a.shape[0]; fx = tgt.shape[1] // a.shape[1]
    return np.repeat(np.repeat(a, fy, 0), fx, 1)

def discrete_cmap(n, base="nipy_spectral", include_zero=True):
    base_c = mpl.cm.get_cmap(base)
    colors = [(0, 0, 0, 1)] if include_zero else []
    colors += list(base_c(np.linspace(0.10 if include_zero else 0, 1.00, n)))
    return mpl.colors.ListedColormap(colors)

# --- Widgets ---
COLORMAPS = ["plasma","viridis","inferno","magma","gray","terrain","cividis","cubehelix"]

patch_dd   = wd.Dropdown(options={f"{y}-{t}-{s.split('_')[-2]}_{s.split('_')[-1]}": (y,t,s) for y,t,s in PATCH_SELECTIONS},
                                 description="Patch:")
band_dd    = wd.Dropdown(description="Band:")
cmap_dd    = wd.Dropdown(options=COLORMAPS, value="plasma", description="Colormap:")
style_dd   = wd.Dropdown(options=["Color + legend",
                                   "Grayscale (no legend)",
                                   "Viridis + legend"],
                                 value="Color + legend", description="Overlay Style:")
ts_slider  = wd.IntSlider(description="Timestamp:")
overlay_rb = wd.RadioButtons(options=["None","Parcels","Labels"], description="Overlay:")
btn_png    = wd.Button(description="Save PNG", button_style="success")
lbl_busy   = wd.Label(value="")
out_fig    = wd.Output()

_state, _cache = {}, {}

# --- UI Helpers ---
def set_busy(on=True): lbl_busy.value = "⏳ Loading…" if on else ""

def update_bands():
    band_dd.options = ["RGB Composite"] + sorted(_state["bands"].keys())
    if band_dd.value not in band_dd.options: band_dd.value = "RGB Composite"
    cmap_dd.disabled = (band_dd.value == "RGB Composite")

# --- Callbacks ---
def on_patch_change(_=None):
    if patch_dd.value is None: return
    set_busy(True)
    y,tile,stem = patch_dd.value; pid = f"{y}_{tile}_{stem}"
    if pid in _cache: bands,parc,lab = _cache[pid]
    else:
        bands,parc,lab = load_patch_disk(y,tile,stem); _cache[pid] = (bands,parc,lab)
    _state.update(bands=bands,parc=parc,lab=lab)
    ts_slider.max = next(iter(bands.values()))["time"].size - 1
    ts_slider.value = 0
    update_bands(); draw(); set_busy(False)

def get_overlay():
    sel = overlay_rb.value
    if sel == "None": return None, None, None
    ds, key = (_state["parc"], "parcels") if sel == "Parcels" else (_state["lab"], "labels")
    arr = ds[key] if "time" not in ds[key].dims else ds[key].isel(time=ts_slider.value)
    data = arr.values

    base_cmap = "viridis" if style_dd.value == "Viridis" else "nipy_spectral"

    if sel == "Labels":
        present = np.unique(data)
        present = present[present != 0]
        n = len(present)
        cmap = discrete_cmap(n, base=base_cmap)
        remap = {c: i+1 for i,c in enumerate(sorted(present))}
        data = np.vectorize(lambda x: remap.get(x,0))(data)
        labels = [crop_map.get(c, str(c)) for c in sorted(present)]
        return data, cmap, labels
    else:
        cmap = discrete_cmap(int(data.max()), base=base_cmap)
        return data, cmap, None

def make_raster():
    b,t = band_dd.value, ts_slider.value; bands = _state["bands"]
    ref_obj = _state["parc"]["parcels"]

    if b == "RGB Composite" and all(k in bands for k in ("B04","B03","B02")):
        # Method CLAHE for natural-looking RGB composites.
        r_raw = resize_to(bands["B04"]["B04"][t].values, ref_obj)
        g_raw = resize_to(bands["B03"]["B03"][t].values, ref_obj)
        b_raw = resize_to(bands["B02"]["B02"][t].values, ref_obj)
        
        rgb_image = np.dstack((r_raw, g_raw, b_raw))
        
        v_min, v_max = np.nanpercentile(rgb_image, (2, 98))
        rgb_clipped = np.clip(rgb_image, v_min, v_max)
        
        rgb_rescaled = (rgb_clipped - v_min) / (v_max - v_min + 1e-9)

        rgb_enhanced = exposure.equalize_adapthist(rgb_rescaled, clip_limit=0.015)
        
        return rgb_enhanced, None

    arr_data = bands[b][b][t].values
    arr_stretched = stretch(arr_data)
    arr = resize_to(arr_stretched, ref_obj)
    return arr, cmap_dd.value

def draw(*_):
    if not _state: return
    out_fig.clear_output(wait=True)
    ov_arr, ov_cmap, labels = get_overlay()
    ras, ras_cmap = make_raster()
    gray_mode = (style_dd.value == "Grayscale (no legend)")

    if gray_mode and ov_arr is not None: ov_cmap = "gray"

    with out_fig:
        fig = plt.figure(figsize=(13, 5))
        has_label_legend = (not gray_mode and labels)
        
        if has_label_legend:
            gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 0.6])
            axL = fig.add_subplot(gs[0, 0])
            axR = fig.add_subplot(gs[0, 1], sharey=axL)
            ax_legend = fig.add_subplot(gs[0, 2])
            ax_legend.axis('off')
        else:
            gs = fig.add_gridspec(1, 2, width_ratios=[1, 1])
            axL = fig.add_subplot(gs[0, 0])
            axR = fig.add_subplot(gs[0, 1], sharey=axL)

        axL.imshow(ras if ras_cmap is None else ras, cmap=ras_cmap or None)
        if ov_arr is not None:
            axL.imshow(ov_arr, cmap=ov_cmap, alpha=.3)
        axL.axis("off")
        axL.set_title(f"{band_dd.value}")
        
        axR.axis("off")
        if ov_arr is not None:
            im = axR.imshow(ov_arr, cmap=ov_cmap)
            
            if has_label_legend:
                handles = [mpatches.Patch(color=ov_cmap(i+1), label=l) for i, l in enumerate(labels)]
                ax_legend.legend(handles=handles, loc='center left', fontsize=8, frameon=False)
            elif not gray_mode and not labels:
                cbar = fig.colorbar(im, ax=axR, fraction=0.046, pad=0.04)
                cbar.ax.tick_params(labelsize=6)
            
            title = overlay_rb.value
            if gray_mode: title += " (grayscale)"
            elif style_dd.value == "Viridis": title += " (viridis)"
            axR.set_title(title)
        else:
            axR.text(.5, .5, "(no overlay selected)", ha="center", va="center", fontsize=9, style='italic', color='gray')
        
        if axR.images: plt.setp(axR.get_yticklabels(), visible=False)

        year, tile, stem = patch_dd.value
        patch_name = f"{stem.split('_')[-2]}_{stem.split('_')[-1]}"
        
        t_idx = ts_slider.value
        time_da = next(iter(_state["bands"].values()))["time"]
        date_str = "Fecha desconocida"
        try:
            date_val = time_da.isel(time=t_idx).values
            date_str = pd.to_datetime(date_val).strftime('%Y-%m-%d')
        except Exception as e:
            print(f"AVISO: No se pudo obtener la fecha. Error: {e}")
        
        fig.suptitle(f"{year} – {tile} – patch {patch_name}  |  t={t_idx} ({date_str})", fontsize=11)
        
        fig.tight_layout(rect=[0, 0, 1, 0.96]); plt.show()

    _state["fig"] = fig

def save_png(_):
    fig=_state.get("fig")
    if fig is None:
        print("⚠️  Draw something first before saving.")
        return
    patch_label = patch_dd.label.replace('/','-')
    fname = OUT_DIR / f"patch_view_{patch_label}_band-{band_dd.value}_t-{ts_slider.value}.png"
    fig.savefig(fname, dpi=300, bbox_inches="tight")
    lbl_busy.value = f"Saved in: {fname}"

# --- Event binding ---
for widget in (patch_dd, band_dd, cmap_dd, ts_slider, overlay_rb, style_dd):
    widget.observe(lambda *_: (set_busy(True), draw(), set_busy(False)), names="value")
patch_dd.observe(on_patch_change, names="value")
btn_png.on_click(save_png)
band_dd.observe(lambda *_: update_bands(), names="value")

# --- UI Assembly ---
display(wd.VBox([
    patch_dd,
    wd.HBox([band_dd, cmap_dd, ts_slider]),
    overlay_rb,
    style_dd,
    wd.HBox([btn_png, lbl_busy]),
    out_fig
]))
on_patch_change()



✓ Crop map loaded (313) from /run/user/1000/gvfs/smb-share:server=10.168.168.61,share=rodrigopg/S4A-CyL_Dataset_2020-2024/mapping_tables/crop_id_mapping.csv
